<center><h1> ChatBot</h1></center>

In the past few years, chatbots in Python have become wildly popular in the tech and business sectors. These intelligent bots are so adept at imitating natural human languages and conversing with humans, that companies across various industrial sectors are adopting them. From e-commerce firms to healthcare institutions, everyone seems to be leveraging this nifty tool to drive business benefits. 

Nowadays almost 30 percent of the tasks are fulfilled by chatbots. Companies use chatbots to provide services like customer support, generating information, etc. With examples like Siri, Alexa it becomes clear how a chatbot can make a difference in our daily lives. 

In this article, we will learn how to make a chatbot in python using the Keras library which implements various machine learning algorithms to generate responses.

## What is a Chatbot?

A chatbot is an AI-based software designed to interact with humans in their natural languages. These chatbots are usually converse via auditory or textual methods, and they can effortlessly mimic human languages to communicate with human beings in a human-like manner. A chatbot is arguably one of the best applications of natural language processing.

A chatbot also known as a chatterbot, bot, artificial agent, etc is basically a software program driven by artificial intelligence that serves the purpose of making a conversation with the user by texts or by speech. Famous examples include Siri, Alexa, etc.

These chatbots are inclined towards performing a specific task for the user. Chatbots often perform tasks like making a transaction, booking a hotel, form submissions, etc. The possibilities with a chatbot are endless with the technological advancements in the domain of artificial intelligence.

Almost 30 percent of the tasks are performed by the chatbots in any company. Companies employ these chatbots for services like customer support, to deliver information, etc. Although the chatbots have come so far down the line, the journey started from a very basic performance. Let’s take a look at the evolution of chatbots over the last few decades.


## History of ChatBots

The first chatbot ever was developed by MIT professor Joseph Weizenbaum in the 1960s. ELIZA, the mother of all chatbots, answered some very simple decision tree questions.
 
In the year 2009, a company called WeChat in China created a more advanced Chatbot. Since its launch, WeChat has conquered the hearts of many users who demonstrate unwavering loyalty to it. It is a highly thriving social media platform.

Today, we have smart AI-powered Chatbots that use natural language processing (NLP) to understand human commands (text and voice) and learn from experience. Think about Apple’s Siri, Amazon’s Alexa, and Microsoft’s Cortana. Aren’t these just wonderful?


### Here are the 5 steps to create a chatbot in Python from scratch:

+ Import and load the data file
- Preprocess data
+ Create training and testing data
+ Build the model
+ Predict the response


### 1. Import and load the data file

In [3]:
# nltk module
import nltk
from nltk.stem.lancaster import LancasterStemmer

# keras module
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd

# other module
import pickle
import random
import json


In [11]:
# create a stemmer
stemmer = LancasterStemmer()

# load intents
# import our chat-bot intents file
import json
with open('intents_robi_en.json') as json_data:
    intents = json.load(json_data)


    
words = []
classes = []
documents = []
ignore_words = []

### 2. Preprocessing Data

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [12]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word into a sentence
        w = nltk.word_tokenize(pattern)
        # add word to our words list
        words.extend(w)
        # add documents to our corpus
        documents.append((w, intent['tag']))

        # add to our classses
        if intent['tag'] not in classes:
            classes.append(intent['tag'])



Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [14]:
# stem and lower case the word and remove duplicates
words =  [stemmer.stem(w.lower()) for w in words ]




# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))



# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

52 documents
8 classes ['Robi_apps', 'balance', 'customer_care', 'goodbye', 'greeting', 'robi', 'social_media_pack', 'thanks']
74 unique stemmed words ["'s", ',', '?', 'account', 'any', 'anyon', 'ap', 'ar', 'awesom', 'bal', 'bil', 'by', 'bye', 'can', 'car', 'chat', 'check', 'cod', 'custom', 'day', 'do', 'download', 'facil', 'fb', 'for', 'get', 'good', 'goodby', 'hello', 'help', 'helplin', 'hi', 'hlw', 'how', 'i', 'imo', 'info', 'internet', 'is', 'lat', 'link', 'loc', 'me', 'med', 'minut', 'my', 'near', 'next', 'nic', 'no', 'numb', 'of', 'off', 'or', 'pack', 'peopl', 'postpaid', 'privileg', 'rob', 'see', 'serv', 'soc', 'spec', 'thank', 'that', 'the', 'ther', 'til', 'tim', 'to', 'what', 'whatsap', 'wher', 'you']


In [22]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "./pickle/robi_en_data.pkl", "wb" ) )

### 3. Create training and testing data

Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [16]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)


# create train and test lists. X - patterns, Y - intents

train_x = list(training[:,0])

train_y = list(training[:,1])

print(train_x[:1], train_y[:1])
len(train_x), len(train_y)

[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] [[0, 1, 0, 0, 0, 0, 0, 0]]


(52, 52)

### 4. Build the model

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [17]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons

# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [18]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               9600      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
Total params: 18,376
Trainable params: 18,376
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [20]:
# Fit the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print("model created")

Epoch 1/200
52/52 [==============================] - 0s 170us/step - loss: 0.0167 - accuracy: 1.0000
Epoch 2/200
52/52 [==============================] - 0s 238us/step - loss: 0.0154 - accuracy: 1.0000
Epoch 3/200
52/52 [==============================] - 0s 170us/step - loss: 0.0200 - accuracy: 1.0000
Epoch 4/200
52/52 [==============================] - 0s 221us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 5/200
52/52 [==============================] - 0s 257us/step - loss: 0.0211 - accuracy: 1.0000
Epoch 6/200
52/52 [==============================] - 0s 188us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 7/200
52/52 [==============================] - 0s 399us/step - loss: 0.0010 - accuracy: 1.0000
Epoch 8/200
52/52 [==============================] - 0s 357us/step - loss: 0.0129 - accuracy: 1.0000
Epoch 9/200
52/52 [==============================] - 0s 380us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 10/200
52/52 [==============================] - 0s 404us/step - loss: 0.0077 - accura

52/52 [==============================] - 0s 313us/step - loss: 0.0132 - accuracy: 1.0000
Epoch 82/200
52/52 [==============================] - 0s 270us/step - loss: 0.0079 - accuracy: 1.0000
Epoch 83/200
52/52 [==============================] - 0s 292us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 84/200
52/52 [==============================] - 0s 273us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 85/200
52/52 [==============================] - 0s 266us/step - loss: 0.0100 - accuracy: 1.0000
Epoch 86/200
52/52 [==============================] - 0s 249us/step - loss: 0.0027 - accuracy: 1.0000
Epoch 87/200
52/52 [==============================] - 0s 185us/step - loss: 0.0069 - accuracy: 1.0000
Epoch 88/200
52/52 [==============================] - 0s 189us/step - loss: 0.0081 - accuracy: 1.0000
Epoch 89/200
52/52 [==============================] - 0s 272us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 90/200
52/52 [==============================] - 0s 276us/step - loss: 0.0042 - accuracy: 

52/52 [==============================] - 0s 256us/step - loss: 0.0107 - accuracy: 1.0000
Epoch 161/200
52/52 [==============================] - 0s 212us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 162/200
52/52 [==============================] - 0s 198us/step - loss: 0.0357 - accuracy: 0.9808
Epoch 163/200
52/52 [==============================] - 0s 235us/step - loss: 0.0137 - accuracy: 1.0000
Epoch 164/200
52/52 [==============================] - 0s 267us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 165/200
52/52 [==============================] - 0s 281us/step - loss: 0.0045 - accuracy: 1.0000
Epoch 166/200
52/52 [==============================] - 0s 275us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 167/200
52/52 [==============================] - 0s 279us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 168/200
52/52 [==============================] - 0s 260us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 169/200
52/52 [==============================] - 0s 219us/step - loss: 0.0301 - a

## 5. Predict the response (Graphical User Interface)

Now to predict the sentences and get a response from the user to let us create a new file ‘chatapp.py’.

We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.

Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model:

In [28]:
# nltk module
import nltk
from nltk.stem.lancaster import LancasterStemmer

# module we need for Tensorflow
import numpy as np
from keras.models import load_model 

# other module
import pandas as pd
import pickle
import random
import json

In [30]:
# create a stemmer
stemmer = LancasterStemmer()

# load intents
# import our chat-bot intents file
import json
with open('intents_robi_en.json') as json_data:
    intents = json.load(json_data)
    
#Use pickle to load  data
data = pickle.load( open( "./pickle/robi_en_data.pkl", "rb" ) )
words = data['words']
classes = data['classes']

# load  the pre-trained model
model = load_model("chatbot_model.h5")
confidence_level = 0.5

To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.

In [31]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return(np.array(bag))




def classify(sentence):
    # Add below two lines for workaround error _thread._local' object has no attribute 'value'
    import keras.backend.tensorflow_backend as tb
    tb._SYMBOLIC_SCOPE.value = True
   
    global model
    global confidence_level
    ERROR_THRESHOLD = confidence_level
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
   
    results = model.predict([input_data],workers=1,verbose=0)[0]
  
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability 
    return return_list

After predicting the class, we will get a random response from the list of intents.

In [32]:
def botResponse(sentence):
    import random
    tagResult = classify(sentence)
    if not tagResult:
        return "Sorry, I do not understand"
    responseList = list(filter(lambda x:x['tag'] == tagResult[0][0],intents['intents']))
    return random.choice(responseList[0]['responses'])

### Test bot response

In [34]:
message='hi'
print("User:",message)
print("Bot:",botResponse(message))

User: hi
Bot: Hi there, how can I help?


In [37]:
message='What is Robi?'
print("User:",message)
print("Bot:",botResponse(message))

User: What is Robi?
Bot: Hello, thanks for choosing Robi. Robi Axiata Limited, DBA Robi is the second largest mobile network operator in Bangladesh. https://www.robi.com.bd/en


In [38]:
message='ok, bye'
print("User:",message)
print("Bot:",botResponse(message))

User: ok, bye
Bot: Bye! Come back again soon.


## Summary
In this Python data science project, we understood about chatbots and implemented a deep learning version of a chatbot in Python which is accurate. You can customize the data according to business requirements and train the chatbot with great accuracy. Chatbots are used everywhere and all businesses is looking forward to implementing bot in their workflow.